<a href="https://colab.research.google.com/github/gcunharodrigues/fcc_book_recomendation/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [157]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-26 20:03:40--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   133MB/s    in 0.2s    

2025-01-26 20:03:40 (133 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [158]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [159]:
# add your code here - consider creating a new cell for each section of code

In [160]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [161]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB


In [326]:
users = df_ratings.groupby('user').agg({'rating': 'count'}).reset_index()
users.columns = ['user', 'rating_count']
users = users[users['rating_count'] >= 200]

In [327]:
isbns = df_ratings.groupby('isbn').agg({'rating': 'count'}).reset_index()
isbns.columns = ['isbn', 'rating_count']
isbns = isbns[isbns['rating_count'] >= 100]

In [328]:
filtered_df_ratings = df_ratings[df_ratings['user'].isin(users['user']) & df_ratings['isbn'].isin(isbns['isbn'])]


In [334]:
merged_df = pd.merge(right=filtered_df_ratings, left=df_books, on='isbn')
merged_df = merged_df.drop_duplicates(['title', 'user'])

In [335]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49136 entries, 0 to 49516
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   isbn    49136 non-null  object 
 1   title   49136 non-null  object 
 2   author  49136 non-null  object 
 3   user    49136 non-null  int32  
 4   rating  49136 non-null  float32
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [336]:
books_features_pivot = merged_df.pivot(
  index='title',
  columns='user',
  values='rating'
).fillna(0)

In [349]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = []
  neighrest_books = []
  recommended_books.append(book)

  data = books_features_pivot.values
  titles = list(books_features_pivot.index.values)

  def title_2_index(title):
    ind = titles.index(title)
    print(title)
    return data[ind,:]

  def index_2_title(ind):
    return titles[ind]

  # Building the model
  model = NearestNeighbors(metric='cosine')
  model.fit(data)

  # Running the model
  idx = title_2_index(book)
  y = model.kneighbors(idx.reshape(1,-1), n_neighbors=7)
  list_recomendations = zip(y[0][0], y[1][0])
  list_recomendations= sorted(
      list(list_recomendations)[1:],
      key=lambda x: x[0],
      reverse=True
  )

  for distance, index in list_recomendations:
    neighrest_books.append([index_2_title(index), distance])

  recommended_books.append(neighrest_books[1:])

  return recommended_books

In [350]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
books

Where the Heart Is (Oprah's Book Club (Paperback))


["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016211],
  ['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

In [351]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Where the Heart Is (Oprah's Book Club (Paperback))
["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
Where the Heart Is (Oprah's Book Club (Paperback))
You passed the challenge! 🎉🎉🎉🎉🎉
